In [25]:
%load_ext autoreload
%autoreload 2
    
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

import time
from datetime import datetime
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys
import os
sys.path.append(os.path.dirname('../.'))

from scripts.load_and_save import (get_dcm_info, get_dcm_vol, vox_size2affine,
                                   load_nii_vol, save_vol_as_nii, raw2nifti)

from ml.get_model import get_model
from ml.utils import get_total_params, load_pretrainned, test_model
from ml.ControllerClass import Controller
from ml.tio_dataset import TioDataset
from ml.metrics import (DICE_Metric, JAC_Metric, SN_Metric, SP_Metric,
                        IOU_Metric, ExponentialLogarithmicLoss)

from ml.models.HessNet import HessBlock, HessNet, HessFeatures, HessUNet, HessUNet2 
from ml.models.unet3d import U_Net
from ml.models.unet2d import U_Net2d
from ml.models.UNETR import UNETR
from ml.models.CBTN import VisionTransformer

In [27]:
DEVICE = 'cuda'
IS2D = 0

In [28]:
if IS2D:
    PATCH_SIZE_TRAIN = (512, 512, 1)
    PATCH_SIZE_TEST = (512, 512, 1)
    OVERLAP_TEST = (0, 0, 0)
else:
    PATCH_SIZE_TRAIN = (64, 64, 64)
    PATCH_SIZE_TEST = (64, 64, 64)#(256, 256, 64)
    OVERLAP_TEST = (16, 16, 16)

train_settings  = {
    "patch_shape" : PATCH_SIZE_TRAIN,
    "patches_per_volume" : 32,
    "patches_queue_length" : 1440,
    "batch_size" : 12,
    "num_workers": 4,
    "sampler": "uniform",
}

test_settings = {
    "patch_shape" : PATCH_SIZE_TEST,
    "overlap_shape" : OVERLAP_TEST,
    "batch_size" : 1,
    "num_workers": 4,
}

data_dir = "/home/msst/Documents/medtech/data/HessData_IXI"
dataset = TioDataset(data_dir,
                 train_settings=train_settings,
                 val_settings=None,#val_settings,
                 test_settings=test_settings)

In [48]:
#model = HessNet(start_scale=[0.8, 0.8, 1.2], device=DEVICE) #HessNet_anisotropic_3, HessNet_anisotropic_4
#model = U_Net(channels=16) #Unet_8ch, Unet_16ch
#model = U_Net2d(channels=16) #Unet_8ch, Unet_16ch

# model = UNETR(in_channels=1, out_channels=1, img_size=PATCH_SIZE_TRAIN,
#               feature_size=16, hidden_size=512,
#               mlp_dim=512, num_heads=4,
#               norm_name='batch')

model = VisionTransformer(img_shape=PATCH_SIZE_TRAIN,
                      patch_shape=(32, 32, 32),
                      in_channels=1,
                      hidden_size=512,
                      transformer_layers=4,
                      attention_heads=8,
                      mlp_dim=512, 
                      drop=0.0)

print("total_params:", get_total_params(model))

total_params: 29015007


In [49]:
controller_config = {
    "loss" : ExponentialLogarithmicLoss(gamma_tversky=0.5, gamma_bce=0.5, lamb=0.5,
                                        freq = 0.1, tversky_alfa=0.5),
    "metric" : DICE_Metric(),
    'device' : DEVICE,
    "model" : model,
    "optimizer_fn" : lambda model: Adam(model.parameters(), lr=3e-5),
    "sheduler_fn": None, #lambda optimizer: StepLR(optimizer, step_size=1, gamma=0.9),
    "is2d" : IS2D,
    'verbose':True
}
controller = Controller(controller_config)

In [50]:
controller.fit(dataset, 60)

Epoch 1/60


  0%|                                                    | 0/43 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (64) must match the size of tensor b (32) at non-singleton dimension 4

In [8]:
#model_name = "Unet3d_16ch_11.10_2"
#model_name = "Unet2d_16ch"
#model_name = "HessNet_isotropic_smartnorm"

#model_name = "UNETR"

In [9]:
#controller.save(f"/home/msst/save_folder/saved_models/{model_name}.pth")
#controller.save_weights(f"/home/msst/save_folder/saved_models/{model_name}_weights.pth")

In [10]:
#path_to_check = f"/home/msst/save_folder/saved_models/{model_name}.pth"
#controller.load(path_to_checkpoint=path_to_check)

In [35]:
sample_index = "020"

path_to_vol = get_path(f"/home/msst/IXI_MRA_work/IXI{sample_index}", key="head")
subject_dict = {'head': tio.ScalarImage(path_to_vol)}
subject = tio.Subject(subject_dict)
subject = tio.transforms.ZNormalization()(subject)

t = time.time()
seg = controller.single_predict(subject, test_settings)
print(time.time() - t)

#dt_string = datetime.now().strftime("%d_%m_%Y_%H:%M")
dir_name = os.path.dirname(path_to_vol)

#path_to_save = f"/home/msst/IXI_MRA_work/IXI{sample_index}/{model_name}_{dt_string}.nii.gz"
path_to_save = f"/home/msst/IXI_MRA_work/IXI{sample_index}/{model_name}.nii.gz"
save_vol_as_nii(seg, subject.head.affine, path_to_save)

2.383925437927246


In [36]:
path_to_GT = get_path(f"/home/msst/IXI_MRA_work/IXI{sample_index}", key="vessels")
GT = tio.ScalarImage(path_to_GT).data
path_to_brain_mask = get_path(f"/home/msst/IXI_MRA_work/IXI{sample_index}", key="brain")
brain_mask = tio.ScalarImage(path_to_brain_mask).data

path_to_save_masked = f"/home/msst/IXI_MRA_work/IXI{sample_index}/{model_name}_mask.nii.gz"
path_to_save_GT_masked = f"/home/msst/IXI_MRA_work/IXI{sample_index}/vessels_mask.nii.gz"
save_vol_as_nii(seg*brain_mask, subject.head.affine, path_to_save_masked)
save_vol_as_nii(GT*brain_mask, subject.head.affine, path_to_save_GT_masked)

In [39]:
import subprocess
import re
path_to_EvaluateSegmentation = '/home/msst/repo/MSRepo/VesselSegmentation/Inference/EvaluateSegmentation'

GT_path = path_to_GT
SEG_path = path_to_save
GT_mask_path = path_to_save_masked
SEG_mask_path = path_to_save_GT_masked

command_output = subprocess.run([f"{path_to_EvaluateSegmentation}",
                                    GT_path, SEG_path], stdout=subprocess.PIPE, text=True)
command_output = command_output.stdout.split('\n')
metrics = ["DICE", "AVGDIST", "SNSVTY"]
metric_dict = {}
for metric in metrics:
    for line in command_output:
        if re.search(metric, line):
            metric_dict.update({metric : line.split('\t')[1][2:]})

command_output = subprocess.run([f"{path_to_EvaluateSegmentation}",
                                    GT_mask_path, SEG_mask_path], stdout=subprocess.PIPE, text=True)
command_output = command_output.stdout.split('\n')
metrics = ["DICE", "AVGDIST", "SNSVTY"]
metric_dict_mask = {}
for metric in metrics:
    for line in command_output:
        if re.search(metric, line):
            metric_dict_mask.update({metric : line.split('\t')[1][2:]})

metric_dict, metric_dict_mask

({'DICE': '0.823599', 'AVGDIST': '2.841585', 'SNSVTY': '0.861042'},
 {'DICE': '0.872858', 'AVGDIST': '0.263038', 'SNSVTY': '0.900524'})

**<h3>GLOBAL TEST**

In [3]:
def run_tests(model_name, path_to_save_models, is2d, test_data_path, epochs):
    if is2d:
        PATCH_SIZE_TRAIN = (512, 512, 1)
        PATCH_SIZE_TEST = (512, 512, 1)
        OVERLAP_TEST = (0, 0, 0)
    else:
        PATCH_SIZE_TRAIN = (64, 64, 64)
        PATCH_SIZE_TEST = (256, 256, 64)
        OVERLAP_TEST = (32, 32, 24)
    
    train_settings  = {
        "patch_shape" : PATCH_SIZE_TRAIN,
        "patches_per_volume" : 64,
        "patches_queue_length" : 1440,
        "batch_size" : 16,
        "num_workers": 4,
        "sampler": "uniform",
    }
    
    test_settings = {
        "patch_shape" : PATCH_SIZE_TEST,
        "overlap_shape" : OVERLAP_TEST,
        "batch_size" : 1,
        "num_workers": 4,
    }
    
    for test in tqdm(os.listdir(test_data_path)):
        dataset = TioDataset(test_data_path + '/' + test,
                  train_settings=train_settings,
                  test_settings=test_settings)
        
        model = get_model(model_name)
        
        controller_config = {
            "loss" : ExponentialLogarithmicLoss(gamma_tversky=0.5, gamma_bce=0.5, lamb=0.5,
                                                freq = 0.1, tversky_alfa=0.5),
            "metric" : DICE_Metric(),
            'device' : 'cuda',
            "model" : model,
            "optimizer_fn" : lambda model: torch.optim.Adam(model.parameters(), lr=0.02),
            "sheduler_fn": lambda optimizer: StepLR(optimizer, step_size=1, gamma=0.9),
            "is2d" : is2d,
            'verbose': False
        }
        controller = Controller(controller_config)
        controller.fit(dataset, epochs)
        controller.save(f"{path_to_save_models}/{model_name}_{test}")

In [4]:
#model_name = 'HessUNet2'
model_name = 'Unet3d_16ch'

run_tests(model_name=model_name, 
          path_to_save_models="/home/msst/save_folder/models_for_tests",
          test_data_path = '/home/msst/Documents/medtech/data/HessData_IXI_test',
          is2d=False,
          epochs=25)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Epoch 1/25
{'mean_loss': 0.7622763651112715}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.7776])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7168])}, {'sample': 'IXI020_0', 'metric1': tensor([0.7513])}]}
Epoch 2/25
{'mean_loss': 0.2930100342879693}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8368])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7817])}, {'sample': 'IXI020_0', 'metric1': tensor([0.8290])}]}
Epoch 3/25
{'mean_loss': 0.2731123532479008}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8429])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7831])}, {'sample': 'IXI020_0', 'metric1': tensor([0.8305])}]}
Epoch 4/25
{'mean_loss': 0.26982324353108805}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8570])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7953])}, {'sample': 'IXI020_0', 'metric1': tensor([0.8501])}]}
Epoch 5/25
{'mean_loss': 0.25067211718608934}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8659])}, {'sample': 'IXI

 50%|█████████████████████▌                     | 1/2 [18:16<18:16, 1096.89s/it]

{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.9343])}, {'sample': 'IXI100_0', 'metric1': tensor([0.8491])}, {'sample': 'IXI020_0', 'metric1': tensor([0.9339])}]}
Epoch 1/25
{'mean_loss': 0.6672985454400381}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.3137])}, {'sample': 'IXI111_0', 'metric1': tensor([0.3520])}, {'sample': 'IXI115_0', 'metric1': tensor([0.4546])}]}
Epoch 2/25
{'mean_loss': 0.2853224550684293}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.7835])}, {'sample': 'IXI111_0', 'metric1': tensor([0.8429])}, {'sample': 'IXI115_0', 'metric1': tensor([0.7320])}]}
Epoch 3/25
{'mean_loss': 0.26316667441278696}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.7689])}, {'sample': 'IXI111_0', 'metric1': tensor([0.8603])}, {'sample': 'IXI115_0', 'metric1': tensor([0.7597])}]}
Epoch 4/25
{'mean_loss': 0.2678287575642268}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.7987])}, {'sample': 'IXI111_0', 'metric1': tensor([0.8774])}, {'sample

100%|███████████████████████████████████████████| 2/2 [36:29<00:00, 1094.53s/it]

{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.8792])}, {'sample': 'IXI111_0', 'metric1': tensor([0.9370])}, {'sample': 'IXI115_0', 'metric1': tensor([0.8047])}]}
